Validation Map

In [2]:
import os
import sys
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../')
import utils.csv as csv
import utils.plot as plot
import utils.shp as shp

In [3]:
PATH='D:\\Deutschland\\FUB\\master_thesis\\data'
LABEL_PATH = os.path.join(PATH, 'ref', 'validation', 'multi_aoi.csv')
SHP_PATH = os.path.join(PATH,'Reference_data','validation', 'aoi_polygons.shp')

In [4]:
aoi = csv.load(LABEL_PATH, 'id')
polygons = shp.load_shp_file(SHP_PATH)

import file D:\Deutschland\FUB\master_thesis\data\Reference_data\validation\aoi_polygons.shp


In [7]:
gdf = pd.merge(polygons, aoi, on='id', how='inner')
class_name = {110:'Spruce', 210:'Silver Fir', 310:'Douglas Fir', 410:'Pine', 
                  600:'Oak', 630:'Red Oak', 710:'Beech', 821:'Sycamore'}
gdf['name'] = gdf['BST1_BA_1'].map(class_name)
gdf.fillna('Others', inplace=True)
gdf

,BST1_BA_1,BST1_BA_2,BST1_BA_3,BST1_BA_4,BST1_BA_5,BST1_BA_6,BST1_BA_7,BST1_BA_8,BST1_BAA_1,BST1_BAA_2,...,area,geometry,Spruce,Silver Fir,Douglas Fir,Pine,Oak,Beech,Sycamore,name
0,410,710,810,630,420,620,859,110,60,25,...,0.000038,"POLYGON ((8.42070 49.06988, 8.42063 49.06990, ...",0,0,0,1,0,1,0,Pine
1,710,410,882,630,821,600,110,0,60,40,...,0.000035,"POLYGON ((8.44339 49.07688, 8.44354 49.07712, ...",0,0,0,1,0,1,0,Beech
2,410,710,630,810,620,110,883,0,45,40,...,0.000031,"POLYGON ((8.42511 49.09021, 8.42539 49.09020, ...",0,0,0,1,0,1,0,Pine
3,410,710,0,0,0,0,0,0,90,10,...,0.000029,"POLYGON ((8.43870 49.07354, 8.43871 49.07354, ...",0,0,0,1,0,0,0,Pine
4,310,210,710,821,851,831,620,110,50,30,...,0.000025,"POLYGON ((7.90083 47.99924, 7.90063 47.99941, ...",0,1,1,0,0,0,0,Douglas Fir
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,851,810,821,979,831,710,600,921,25,25,...,0.000001,"POLYGON ((8.93830 48.59221, 8.93843 48.59236, ...",0,0,0,0,0,0,0,Others
190,921,831,979,874,810,110,600,710,70,20,...,0.000001,"POLYGON ((8.93760 48.59478, 8.93785 48.59498, ...",0,0,0,0,0,0,0,Others
191,710,821,110,980,950,210,0,0,50,30,...,0.000001,"POLYGON ((7.92569 47.99797, 7.92569 47.99800, ...",0,0,0,0,0,1,1,Beech
192,110,210,410,710,920,510,310,930,75,15,...,0.000001,"POLYGON ((8.95481 48.58559, 8.95480 48.58587, ...",1,0,0,0,0,0,0,Spruce


In [8]:
# Karlsruhe
aoi_1 = gdf[gdf['Location'] == 'Hardtwald_pine_beech_redoak']
# Stuttgart
aoi_2 = gdf[gdf['Location'] == 'schoenbuch_beech_oak_mixture']
# Freiburg
aoi_3 = gdf[gdf['Location'] == 'Schwarzwald_spruce_silverfir_douglasfir']
# dict of aoi
areas = {'Hardtwald':aoi_1, 'Schoenbuch':aoi_2, 'Schwarzwald':aoi_3}
# draw map
for area, gdf in areas.items():
    plot.draw_color_map(gdf, area, 'Origin')
    plot.draw_symbol_map(gdf, area, 'Origin')

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

AOI SHP Generation

In [4]:
from geopandas.tools import sjoin
file_path = os.path.join(PATH, 'Reference_data', 'bw_all_polygons', 'buffered_wgs_bw_polygons.shp')
aoi_path = os.path.join(PATH, 'Reference_data', 'validation', 'Objectbased_classification_regression_validation2.shp')
out_path = os.path.join(PATH, 'Reference_data', 'validation', 'aoi_polygons.shp')

In [5]:
polygons = shp.load_shp_file(file_path)
aoi = shp.load_shp_file(aoi_path)
aoi.drop(columns=['id'], inplace=True)
aoi = shp.reproject(aoi)

import file D:\Deutschland\FUB\master_thesis\data\Reference_data\bw_all_polygons\buffered_wgs_bw_polygons.shp
import file D:\Deutschland\FUB\master_thesis\data\Reference_data\validation\Objectbased_classification_regression_validation2.shp
Reproject to EPSG:4326


In [6]:
assert polygons.crs == aoi.crs, "Coordinate Reference Systems (CRS) must match"
val_polygons = sjoin(polygons, aoi, how='inner', op='intersects')
val_polygons.drop(columns=['index_right'], inplace=True)
polygons_utm = val_polygons.to_crs(epsg=25832)
val_polygons['area'] = polygons_utm.geometry.area / 1e6
val_polygons.sort_values(by='area', ascending=False, inplace=True)
output = val_polygons.head(195)
shp.export_shp_file(output, out_path)
shp.export_csv_reference(output, LABEL_PATH)

export file D:\Deutschland\FUB\master_thesis\data\Reference_data\validation\aoi_polygons.shp
export file D:\Deutschland\FUB\master_thesis\data\ref\validation\reference_aoi.csv


d:\LenovoSoftstore\Anaconda\envs\geo\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
